In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#read in reduced csv from files on local machine
df = pd.read_csv('/Users/devpatel/Desktop/nashville_reviews.csv')
from datasets import load_dataset

nash_reviews = load_dataset('csv', data_files='/Users/devpatel/Desktop/nashville_reviews.csv')



Found cached dataset csv (/Users/devpatel/.cache/huggingface/datasets/csv/default-ac9cc7615352b7e9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
nash_reviews['train']

Dataset({
    features: ['Unnamed: 0', 'text', 'stars'],
    num_rows: 452253
})

In [3]:
nash_reviews['train'][100]

{'Unnamed: 0': 885,
 'text': 'A bit pretentious... unaccommodating for even severe allergies (anaphylaxis).  Different for the the sake of being different rather than being good... hmmm I have 2 short ribs and 10 orders ... lets cut them on the bias!  Theyll be fatty tough and grisly!',
 'stars': 2}

In [9]:
nash_reviews.keys()

dict_keys(['train'])

In [6]:
#drop unamed column from dictionary
del nash_reviews['Unnamed: 0']

KeyError: 'Unnamed: 0'

In [10]:
nash_reviews['train'].pop('Unnamed: 0')

AttributeError: 'Dataset' object has no attribute 'pop'

In [11]:
type(nash_reviews['train'])

datasets.arrow_dataset.Dataset

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = nash_reviews.map(tokenize_function, batched=True)

Map:   0%|          | 0/452253 [00:00<?, ? examples/s]

In [54]:
tokenized_datasets= tokenized_datasets.remove_columns('Unnamed: 0')

In [55]:
tokenized_datasets['train']

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 452253
})

In [56]:
#get number of rows in dataset
len(tokenized_datasets['train'])

452253

In [57]:
#rename stars column to labels
tokenized_datasets=tokenized_datasets.rename_column("stars", "labels")

ValueError: Original column name stars not in the dataset. Current columns in the dataset: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [85]:
traindata =tokenized_datasets['train'].shuffle(seed=42).select(range(300))
valdata = tokenized_datasets['train'].shuffle(seed=42).select(range(300, len(tokenized_datasets['train']-len(traindata))))

Loading cached shuffled indices for dataset at /Users/devpatel/.cache/huggingface/datasets/csv/default-ac9cc7615352b7e9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-a48ec87a0860d24a.arrow
Loading cached shuffled indices for dataset at /Users/devpatel/.cache/huggingface/datasets/csv/default-ac9cc7615352b7e9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-a48ec87a0860d24a.arrow


TypeError: unsupported operand type(s) for -: 'Dataset' and 'int'

In [86]:
traindata

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300
})

In [87]:
#change the labels make them 0-4 instead of 1-5
for i in range(len(traindata['labels'])):
    if traindata['labels'][i] == 1:
        traindata['labels'][i] = 0
    elif traindata['labels'][i] == 2:
        traindata['labels'][i] = 1
    elif traindata['labels'][i] == 3:
        traindata['labels'][i] = 2
    elif traindata['labels'][i] == 4:
        traindata['labels'][i] = 3
    elif traindata['labels'][i] == 5:
        traindata['labels'][i] = 4

In [93]:
traindata['labels'][traindata['labels'] == 1] = 0
traindata['labels'][traindata['labels'] == 2] = 1
traindata['labels'][traindata['labels'] == 3] = 2
traindata['labels'][traindata['labels'] == 4] = 3
traindata['labels'][traindata['labels'] == 5] = 4

False

In [74]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [75]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [81]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

device(type='cpu')

In [77]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [78]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [79]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=traindata,
    eval_dataset=valdata,
    compute_metrics=compute_metrics,
)

In [80]:
trainer.train()

/Users/devpatel/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df

In [ ]:
#convert df to dictionary
dict = df.to_dict()

In [ ]:
dict.keys()


In [ ]:
dict.values()

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
#apply preprocessing to the dataset text column

In [ ]:
#conver dataframe to dictionary use text as key and stars as value
dict = df.to_dict()

In [ ]:
dict['text']

In [ ]:
tokenized_rev = dict.map(preprocess_function, batched=True)

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
df['text']

In [ ]:
df['stars']

In [ ]:
sentiment_score(df['text'].iloc[3])


In [ ]:
df['sentiment'] = df['text'].apply(sentiment_score)

In [ ]:
df